# Choosing a location for an oil well


This analysis is carried out for the mining company "Company X". Based on the data, we need to decide where to drill a new well.

We were provided with oil samples in three regions: in each 10,000 fields, where we measured the quality of oil and the volume of its reserves. Using machine learning, we will determine the region where mining will bring the greatest profit.

<b>Steps to choose a location:</b>

- Oil fields are searched in a selected region, and the the features are determined for each;
- The model is built and the volume of reserves is estimated;
- Fields with the highest value estimates are selected. The number of fields depends on the company's budget and the cost of developing of one well;
- Profit is equal to the total profit of the selected fields

<b>Conditions for a location:</b>
- When the region is explored, 500 areas are estimated, and after analysis the best 200 areas are selected for development.
- The budget for well development in one region is 10 billion rubles.
- At current prices, one barrel of raw oil brings 450 rubles of income. The income from each unit is 450 thousand rubles, because the volume is specified in thousands of barrels.
- After assessing the risks, it is necessary to leave only those regions where the probability of losses is less than 2.5%. Among them the region with the highest average profit is selected.

<b>Input data:</b>
- /datasets/geo_data_0.csv. - Region #0 data
- /datasets/geo_data_1.csv. - Region #1 data
- /datasets/geo_data_2.csv. - Region #2 data
- id - well unique identifier
- f0, f1, f2 - three well features
- product - reserves volume in the well (thousand barrels)

Downloading and preparing data


Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import math


<br>
Loading data files and creating a list (for further convenience of working with cycles). We can not combine the files - we need to analyze each region separately.


In [ ]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')
df_list = [df0, df1, df2]



<br>
Let's see what the data looks like.

In [ ]:
for df, j in zip(df_list, range(3)):
    display(f'----- Данные Региона {j} -----', df.head(3))

'----- Данные Региона 0 -----'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


'----- Данные Региона 1 -----'

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


'----- Данные Региона 2 -----'

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


<br>
Exploring null values and data types.

In [ ]:
for df, j in zip(df_list, range(3)):
    display(f'----- Данные Региона {j} -----')
    display(df.info())

'----- Данные Региона 0 -----'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

'----- Данные Региона 1 -----'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

'----- Данные Региона 2 -----'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<br>
Посмотрим на разброс данных и аномальные значения.

In [ ]:
for df, j in zip(df_list, range(3)):
    display(f'----- Данные Региона {j} -----', df.describe().T)

'----- Данные Региона 0 -----'

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.500419,0.871832,-1.408605,-0.072580,0.502360,1.073581,2.362331
f1,100000.0,0.250143,0.504433,-0.848218,-0.200881,0.250252,0.700646,1.343769
f2,100000.0,2.502647,3.248248,-12.088328,0.287748,2.515969,4.715088,16.003790
product,100000.0,92.500000,44.288691,0.000000,56.497507,91.849972,128.564089,185.364347


'----- Данные Региона 1 -----'

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,1.141296,8.965932,-31.609576,-6.298551,1.153055,8.621015,29.421755
f1,100000.0,-4.796579,5.119872,-26.358598,-8.267985,-4.813172,-1.332816,18.734063
f2,100000.0,2.494541,1.703572,-0.018144,1.000021,2.011479,3.999904,5.019721
product,100000.0,68.825000,45.944423,0.000000,26.953261,57.085625,107.813044,137.945408


'----- Данные Региона 2 -----'

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.002023,1.732045,-8.760004,-1.162288,0.009424,1.158535,7.238262
f1,100000.0,-0.002081,1.730417,-7.084020,-1.174820,-0.009482,1.163678,7.844801
f2,100000.0,2.495128,3.473445,-11.970335,0.130359,2.484236,4.858794,16.739402
product,100000.0,95.000000,44.749921,0.000000,59.450441,94.925613,130.595027,190.029838


<br>
Processiog duplicates.

In [ ]:
for df, j in zip(df_list, range(3)):
    print(f'Дублей строк в файле № {j}:', df.duplicated().sum(), 
          f' -  Дублей номеров скважин:', df['id'].duplicated().sum())

Дублей строк в файле № 0: 0  -  Дублей номеров скважин: 10
Дублей строк в файле № 1: 0  -  Дублей номеров скважин: 4
Дублей строк в файле № 2: 0  -  Дублей номеров скважин: 4


Completely duplicated lines were not found in the data. Despite the fact that there are duplicated well numbers, we will not delete these lines. Presumably, at these points measurements were carried out several times. Thus, even if the measurement data were changed, they are of the same value for us.

<br>
Let's remove the column with the number of wells, there is no valuable information in it for the project.

In [ ]:
for df in df_list:
    df.drop(columns = 'id', inplace=True)

### Conclusion to Part 1

We have 3 clean, informative datasets suitable for building a linear regression model. The data types are normal, no null values, duplicates, or outliers were detected.

## Train and validation the model

Dividing the data into features (f1, f2, f3) and the target feature (product), applying the linear regression model. The results of model predictions and current data will be saved into separate variables for further manipulation.

In [ ]:
state = np.random.RandomState(1)

target_mean = []  # average value of current well resources
predictions_mean = []  # average value of predicted well resources
rmse = []  # deviation metric
target_data = pd.DataFrame()  # current well resources
predictions_data = pd.DataFrame()  # predicted well resources

for df, i in zip(df_list, range(3)):
    features = df.drop(columns='product')
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=state)

    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    score = np.sqrt(mean_squared_error(target_valid, predicted))  # calculate RMSE metric

    predictions_data[i] = predicted  # write data on dataset i to the prepared empty DataFrame
    target_data[i] = target_valid.reset_index(drop=True)  # similar operation for target dataset i
    target_mean.append(target_valid.mean())  # calculate the current average value of the well resource of region i
    predictions_mean.append(predicted.mean())  # in the same way with model predictions
    rmse.append(score.mean())


In [ ]:
result = pd.DataFrame(
    data=[target_mean, predictions_mean, rmse],
    index=['Target Mean', 'Predictions Mean', 'RMSE'],
    columns=['Region 1', 'Region 2', 'Region 3'])
result

,Region 1,Region 2,Region 3
Target Mean,92.423414,69.030000,95.281354
Predictions Mean,92.492625,69.035456,95.017628
RMSE,37.742587,0.892854,39.798777


### Conclusion to Part 2

The average current and predicted reserves are very close. At the same time, in the second region, the average reserves are significantly behind the other two. Given the minimal error in the same region, these reserves may indeed be so.

## Preparation for profit calculation

Saving all input data to constants.

In [ ]:
BUDGET = 10000000000 # budget for development in one region, rubles.
BARREL_INCOME = 450 # profit per barrel, rubles.
PRODUCT_INCOME = 450000 # profit per product (1kbbl), rub.
TOTAL_WELL = 500 # number of mines to explore when developing new region
BEST_WELL = 200 # number of mines with the best indexes which are allowed to develop

<br>
Calculation of the minimum resource of the product of one well to reach break-even point.

In [ ]:
min_well_product = BUDGET / BEST_WELL / PRODUCT_INCOME
min_well_product

111.11111111111111

<br>

Calculation of the difference between the average current resources of each region and the breakeven indicator.

In [ ]:
difference = []

for region in result.columns:
    difference.append(result.loc['Target Mean', region] - min_well_product)
result.append(pd.DataFrame(data=[difference], index=['Difference (Target - Min Well)'], columns=result.columns))

,Region 1,Region 2,Region 3
Target Mean,92.423414,69.030000,95.281354
Predictions Mean,92.492625,69.035456,95.017628
RMSE,37.742587,0.892854,39.798777
Difference (Target - Min Well),-18.687697,-42.081112,-15.829757


### Conclusion to Part 3

In all regions, the average product reserves are below the required level to break even (111,000 barrels). At the same time, in the second region this indicator is significantly (2-3 times) lower than in the first and third regions.

## Calculation of profits and risks 

Creatin a formula to calculate the profit for one region in the development of 200 best wells.

In [ ]:
def revenue(target, predictions, count):
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    return PRODUCT_INCOME * selected.sum() - BUDGET

<br>
Using the bootstrap method, I'm estimate the possible profits based on the condition "500 mines are explored in the region, and the best 200 are taken into development".

In [ ]:
final_results = [] # container to collect information for the summary table

for i in range(3):
    values = [] # container to collect profit figures 
    for j in range(1000):
        sampled_target = target_data[i].sample(TOTAL_WELL, replace=True, random_state=state)
        sampled_pred = predictions_data[i][sampled_target.index]
        values.append(revenue(sampled_target, sampled_pred, BEST_WELL))  # collection of revenue figures
        
    values = pd.Series(values)
    revenue_mean = values.mean()/10**9
    risk = (values < 0).mean() * 100
    interval = (values.quantile(0.025), values.quantile(0.975))
    final_results.append((f'{round(revenue_mean, 2)} billion.', f'{round(risk, 2)} %', interval))


final_results = pd.DataFrame(data=final_results,
                             columns=['Mean Revenue', 'Risk', 'Revenue Interval'],
                             index=['Region 0', 'Region 1', 'Region 2'])
final_results

,Mean Revenue,Risk,Revenue Interval
Region 0,0.46 млрд.,5.0 %,"(-64744999.0024897, 1011105081.2977668)"
Region 1,0.51 млрд.,0.9 %,"(58840392.28345285, 941519779.5067781)"
Region 2,0.43 млрд.,6.0 %,"(-80230497.36746167, 951400734.5322126)"


### Conclusion to Part 4

The profit margins for all regions are not very different at the scale of the project. Despite the fact that the region number 2 promises better profits, it is not recommended to take in the development, because the average stock of the product in it is about 35% lower than in other regions.

Based on the level of risk, potential profit and its confidence interval, region #1 is recommended for development.